In [ ]:
# -------------------------------------------
# 문제정의
# -------------------------------------------
# 주어진 제품 및 매장 특성들을(X값, 컬럼)기반으로 판매 금액(Item_Outlet_Sales)을 예측하기

In [ ]:
# -------------------------------------------
# 라이브러리 가져오기
# -------------------------------------------

In [1]:
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")



SyntaxError: unterminated string literal (detected at line 2) (1977136452.py, line 2)

In [ ]:
# -------------------------------------------
# EDA
# -------------------------------------------

In [ ]:
# train.head()
# train.tail()
train.sample(5)

In [ ]:
train.shape, test.shape

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.describe(include='O')

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train.isnull().sum().sum()

In [ ]:
test.isnull().sum().sum()

In [ ]:
# -------------------------------------------
# 데이터 전처리
# -------------------------------------------

In [ ]:
# y값 분할
target = train.pop('Item_Outlet_Sales')
target

In [ ]:
# 결측값 채우기(무게 : 최소값, 사이즈 : 평균값)

train['Item_Weight'] = train['Item_Weight'].fillna(train['Item_Weight'].min())
train['Outlet_Size'] = train['Outlet_Size'].fillna(train['Outlet_Size'].mode()[0])

test['Item_Weight'] = test['Item_Weight'].fillna(test['Item_Weight'].min())
test['Outlet_Size'] = test['Outlet_Size'].fillna(test['Outlet_Size'].mode()[0])

In [ ]:
train.describe()

In [ ]:
# 레이블인코딩

from sklearn.preprocessing import LabelEncoder

cols = train.columns[train.dtypes == object]

for col in cols:
    le = LabelEncoder()
    le.fit(pd.concat([train[col],test[col]] ,axis=0)) # 행합치기
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

In [ ]:
train

In [ ]:
# 컬럼제거

In [ ]:
train.drop('Item_Identifier', axis=1, inplace=True)
test.drop('Item_Identifier', axis=1, inplace=True)

In [ ]:
# -------------------------------------------
# 검증 데이터 분할 train - val(8,2)
# -------------------------------------------

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    train,
    y_train,
    test_size=0.2,
    random_state=0
)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
# -------------------------------------------
# 학습 및 평가(MSE, MAE, R2...)
# -------------------------------------------

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression() # 모델 가져오기
lr.fit(X_train, y_train) # 학습
y_pred = lr.predict(X_val) # 예측
y_pred

In [ ]:
# 평가
from sklearn.metrics import mean_squared_error # MSE(평균 제곱 오차)
from sklearn.metrics import mean_absolute_error # MAE(평균 절대 오차)
from sklearn.metrics import root_mean_squared_error # RMSE(제곱 평균 오차 루트)
from sklearn.metrics import r2_score #R2Score (결정계수)

lr_mse = mean_squared_error(y_val, y_pred)
lr_mae = mean_absolute_error(y_val, y_pred)
lr_rmse = root_mean_squared_error(y_val, y_pred)
lr_r2 = r2_score(y_val, y_pred)

print("MSE : ", lr_mse) # 오차를 제곱해 평균을 낸 값 - 값 클수록 예측이 실제와 다름
print("MAE : ", lr_mae) # 오차를 절대값으로 계산 - 값 클수록 예측이 실제와 다름
print("RMSE : ", lr_rmse) # 
print("R2 : ", lr_r2) # 

# MSE / MAE / RMSE 상대적인 기준
# 평균적으로 Y 평균값의 10% 이하 : 준수한 성능
# 평균적으로 Y 평균값의 10-30% : 보통
# Y 평균값의 30% 이상 : 낮은 성능
# 시험점수(0-100) 예측 -> RMSE = 10 - 문제
# 아파트가격(30평 - 평균5억) 예측 -> RMSE=1000만원(2%) -> 오차작음(괜찮은 모델)

# R2(결정계수)
# 0.7 이상 : 좋은 모델
# 0.5 - 0.7 : 보통
# 0.3 - 0.5 : 낮은모델

In [ ]:
# LIGHTGBM 학습

In [ ]:
# LIGHTGBM 학습
import lightgbm as lgb
model = lgb.LGBMRegressor(random_state=0, verbose=-1)
model.fit(X_train, y_train)
y_pred_2 = model.predict(X_val)

In [ ]:
lgm_mse = mean_squared_error(y_val, y_pred_2)
lgm_mae = mean_absolute_error(y_val, y_pred_2)
lgm_rmse = root_mean_squared_error(y_val, y_pred_2)
lgm_r2 = r2_score(y_val, y_pred_2)

print("MSE : ", lgm_mse) # 오차를 제곱해 평균을 낸 값 - 값 클수록 예측이 실제와 다름
print("MAE : ", lgm_mae) # 오차를 절대값으로 계산 - 값 클수록 예측이 실제와 다름
print("RMSE : ", lgm_rmse) # 
print("R2 : ", lgm_r2) # 

In [ ]:
# -------------------------------------------
# 파일로 저장
# -------------------------------------------
pred = model.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv("result.csv, index = False)